# Volumetric Reconstruction Pipeline

### Dependencies
* OpenCV, NumPy, Matplotlib
* Data directory structure: `data/cam1/`, `data/cam2/`, etc.

In [1]:
import cv2
import numpy as np
import os

# Shared Configuration
CAM_IDS = [1, 2, 3, 4]
CHECKERBOARD_SIZE = (6, 8)  # Defined in checkerboard.xml
SQUARE_SIZE = 25             # mm

def get_config_path(cam_id):
    return f'data/cam{cam_id}/config.xml'

## Task 1: Calibration (Kian)
Focus: Intrinsics from `intrinsics.avi` and Extrinsics from `checkerboard.avi`.

In [ ]:
import cv2
import numpy as np
import xml.etree.ElementTree as ET

# Projects a grid onto the image plane using a perspective transform derived from the corners
def project_grid_perspective(four_corners, cols, rows):
    src_ideal = np.array([
        [0, 0], [cols-1, 0], [cols-1, rows-1], [0, rows-1]
    ], dtype=np.float32)
    
    dst_current = np.array(four_corners, dtype=np.float32)
    H_matrix = cv2.getPerspectiveTransform(src_ideal, dst_current)
    grid_x, grid_y = np.meshgrid(np.arange(cols), np.arange(rows))
    ideal_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T.astype(np.float32).reshape(-1, 1, 2)
    
    return cv2.perspectiveTransform(ideal_points, H_matrix)

# Warps the image region to a flat view to refine corner detection, then projects points back
def refine_corners_via_warping(image_gray, four_corners, cols, rows):
    w_top = np.linalg.norm(four_corners[0] - four_corners[1])
    w_bot = np.linalg.norm(four_corners[3] - four_corners[2])
    h_left = np.linalg.norm(four_corners[0] - four_corners[3])
    h_right = np.linalg.norm(four_corners[1] - four_corners[2])
    
    dst_width = int(max(w_top, w_bot))
    dst_height = int(max(h_left, h_right))
    padding = 50 
    
    dst_corners = np.array([
        [padding, padding],
        [padding + dst_width, padding],
        [padding + dst_width, padding + dst_height],
        [padding, padding + dst_height]
    ], dtype=np.float32)
    
    M = cv2.getPerspectiveTransform(np.array(four_corners, dtype=np.float32), dst_corners)
    M_inv = np.linalg.inv(M) 
    warped_image = cv2.warpPerspective(image_gray, M, (dst_width + 2*padding, dst_height + 2*padding))
    
    grid_x, grid_y = np.meshgrid(
        np.linspace(padding, padding + dst_width, cols),
        np.linspace(padding, padding + dst_height, rows)
    )
    
    warped_guesses = np.vstack([grid_x.ravel(), grid_y.ravel()]).T.astype(np.float32)
    warped_guesses = np.ascontiguousarray(warped_guesses).reshape(-1, 1, 2)
    
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 40, 0.001)
    refined_warped = cv2.cornerSubPix(warped_image, warped_guesses, (5, 5), (-1, -1), criteria)
    
    return cv2.perspectiveTransform(refined_warped, M_inv)

# Interactive GUI with Magnifying Glass to allow users to move 4 corners
def select_corners_interface(image_gray, cols, rows, initial_handles=None):
    CLICK_SENSITIVITY = 20
    HANDLE_RADIUS = 1
    GRID_RADIUS = 1
    
    height, width = image_gray.shape
    
    if initial_handles is not None:
        handles = [np.array(pt, dtype=np.float32) for pt in initial_handles]
    else:
        margin_horizontal, margin_vertical = width // 4, height // 4
        handles = [
            np.array([margin_horizontal, margin_vertical], dtype=np.float32),
            np.array([width - margin_horizontal, margin_vertical], dtype=np.float32),
            np.array([width - margin_horizontal, height - margin_vertical], dtype=np.float32),
            np.array([margin_horizontal, height - margin_vertical], dtype=np.float32)
        ]
    
    state = {
        "drag_index": None,
        "mouse_pos": (width // 2, height // 2),
        "handles": handles
    }
    
    window_name = "Manual Calibration - Drag corners. Press 'c' to confirm"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, 1080, int(1080 * (height/width)))

    def mouse_callback(event, x, y, flags, param):
        state["mouse_pos"] = (x, y)
        if event == cv2.EVENT_LBUTTONDOWN:
            for idx, pt in enumerate(state["handles"]):
                if np.linalg.norm(pt - [x, y]) < CLICK_SENSITIVITY:
                    state["drag_index"] = idx
                    break
        elif event == cv2.EVENT_MOUSEMOVE and state["drag_index"] is not None:
            state["handles"][state["drag_index"]] = np.array([x, y], dtype=np.float32)
        elif event == cv2.EVENT_LBUTTONUP:
            if state["drag_index"] is not None:
                corner = np.array([[[x, y]]], dtype=np.float32)
                corner = np.ascontiguousarray(corner)
                state["handles"][state["drag_index"]] = corner[0, 0]
                state["drag_index"] = None

    cv2.setMouseCallback(window_name, mouse_callback)
    
    display_color = cv2.cvtColor(image_gray, cv2.COLOR_GRAY2BGR)
    colors = [(0, 0, 255), (255, 0, 0), (0, 255, 0), (0, 255, 255)]
    
    while True:
        temp_img = display_color.copy()
        
        corner_points = np.array(state["handles"], np.int32).reshape((-1, 1, 2))
        cv2.polylines(temp_img, [corner_points], True, (255, 255, 0), 1)
        
        grid_points = project_grid_perspective(state["handles"], cols, rows)
        if grid_points is not None:
            for point in grid_points:
                cv2.circle(temp_img, tuple(point[0].astype(int)), GRID_RADIUS, (0, 255, 0), -1)
                
        for idx, point in enumerate(state["handles"]):
            cv2.circle(temp_img, tuple(point.astype(int)), HANDLE_RADIUS, colors[idx], -1)
            
        mouse_x = np.clip(state["mouse_pos"][0], 0, width - 1)
        mouse_y = np.clip(state["mouse_pos"][1], 0, height - 1)
        pad_size = 30
        
        pad_img = cv2.copyMakeBorder(temp_img, pad_size, pad_size, pad_size, pad_size, cv2.BORDER_REPLICATE)
        cx, cy = mouse_x + pad_size, mouse_y + pad_size
        patch = pad_img[cy-pad_size:cy+pad_size, cx-pad_size:cx+pad_size]
        
        zoom_size = pad_size * 4
        patch_zoomed = cv2.resize(patch, (zoom_size, zoom_size), interpolation=cv2.INTER_NEAREST)
        
        cv2.line(patch_zoomed, (zoom_size//2, 0), (zoom_size//2, zoom_size), (255, 255, 255), 1)
        cv2.line(patch_zoomed, (0, zoom_size//2), (zoom_size, zoom_size//2), (255, 255, 255), 1)
        cv2.rectangle(patch_zoomed, (0,0), (zoom_size-1, zoom_size-1), (255, 255, 255), 2)
        
        if mouse_x > width - zoom_size - 20 and mouse_y < zoom_size + 20:
            temp_img[10:10+zoom_size, 10:10+zoom_size] = patch_zoomed
        else:
            temp_img[10:10+zoom_size, width-zoom_size-10:width-10] = patch_zoomed

        cv2.imshow(window_name, temp_img)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            try:
                refined_corner_points = refine_corners_via_warping(image_gray, state["handles"], cols, rows)
                cv2.destroyWindow(window_name)
                return refined_corner_points, np.array(state["handles"])
            except Exception as e:
                print(f"Refinement failed: {e}")
                
# Calibrate camera intrinsics using intrinsics.avi and checkerboard.xml.
def calibrate_intrinsics(cam_id):
    xml_path = "data/checkerboard.xml"
    tree = ET.parse(xml_path)
    root = tree.getroot()
    cols = int(root.find("CheckerBoardWidth").text)
    rows = int(root.find("CheckerBoardHeight").text)
    square_size = float(root.find("CheckerBoardSquareSize").text)

    object_points = np.zeros((rows * cols, 3), np.float32)
    object_points[:, :2] = np.mgrid[0:cols, 0:rows].T.reshape(-1, 2)
    object_points *= square_size

    video_path = f"data/cam{cam_id}/intrinsics.avi"
    cap = cv2.VideoCapture(video_path)
    
    objpoints = []
    imgpoints = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(1, frame_count // 10)
    
    for i in range(0, frame_count, step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            continue
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        found, corners = cv2.findChessboardCorners(gray, (cols, rows), None)
        if found:
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            objpoints.append(object_points)
            imgpoints.append(corners2)
            
    cap.release()
    if len(objpoints) < 3:
        raise RuntimeError("Not enough valid frames for calibration.")
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return mtx, dist    

# Calibrate camera extrinsics using checkerboard.avi and manual corner selection.
def calibrate_extrinsics(cam_id, mtx, dist):
    xml_path = "data/checkerboard.xml"
    tree = ET.parse(xml_path)
    root = tree.getroot()
    cols = int(root.find("CheckerBoardWidth").text)
    rows = int(root.find("CheckerBoardHeight").text)
    square_size = float(root.find("CheckerBoardSquareSize").text)

    object_points = np.zeros((rows * cols, 3), np.float32)
    object_points[:, :2] = np.mgrid[0:cols, 0:rows].T.reshape(-1, 2)
    object_points *= square_size
    
    initial_handles = None
    config_path = get_config_path(cam_id)
    if os.path.exists(config_path):
        fs_read = cv2.FileStorage(config_path, cv2.FILE_STORAGE_READ)
        corner_node = fs_read.getNode("manual_corners")
        if not corner_node.empty():
            initial_handles = corner_node.mat()
        fs_read.release()

    video_path = f"data/cam{cam_id}/checkerboard.avi"
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count // 2)
    ret, frame = cap.read()
    if not ret:
        cap.release()
        raise RuntimeError(f"Could not read checkerboard frame for Cam {cam_id}.")
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cap.release()
    
    print(f"--- Extrinsic Calibration: Camera {cam_id} ---")
    
    corners2d, handles = select_corners_interface(gray, cols, rows, initial_handles)
    
    ret, rvec, tvec = cv2.solvePnP(object_points, corners2d, mtx, dist)
    return rvec, tvec, handles

# Saves all calibration parameters to an XML file for future use
def save_all_params(cam_id, mtx, dist, rvec, tvec, manual_corners):
    fs = cv2.FileStorage(get_config_path(cam_id), cv2.FILE_STORAGE_WRITE)
    fs.write("camera_matrix", mtx)
    fs.write("distortion_coefficients", dist)
    fs.write("rotation_vector", rvec)
    fs.write("translation_vector", tvec)
    fs.write("manual_corners", manual_corners)
    fs.release()

# Projects the world origin using matching intrinsics and extrinsics.
def visualize_checkerboard_start(cam_id, mtx, dist, rvec, tvec):
    xml_path = "data/checkerboard.xml"
    tree = ET.parse(xml_path)
    root = tree.getroot()
    square_size = float(root.find("CheckerBoardSquareSize").text)
    axis_length = square_size * 3
    axis_points = np.float32([
        [0, 0, 0],
        [axis_length, 0, 0],
        [0, axis_length, 0],
        [0, 0, -axis_length]
    ])
    axis_colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0)]
    
    imgpts, _ = cv2.projectPoints(axis_points, rvec, tvec, mtx, dist)
    imgpts = imgpts.reshape(-1, 2).astype(int)

    video_path = f"data/cam{cam_id}/checkerboard.avi"
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, 5) 
    ret, frame = cap.read()
    cap.release()

    if ret:
        origin = tuple(imgpts[0])
        pt_x = tuple(imgpts[1])
        pt_y = tuple(imgpts[2])
        pt_z = tuple(imgpts[3])
        
        cv2.line(frame, origin, pt_x, axis_colors[0], 2)
        cv2.line(frame, origin, pt_y, axis_colors[1], 2)
        cv2.line(frame, origin, pt_z, axis_colors[2], 2)
        
        cv2.circle(frame, origin, 3, (255, 255, 255), -1)
        
        cv2.imshow(f"Camera {cam_id} Axes", frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print(f"Error: Could not load frame for Camera {cam_id}")

## Task 2: Background Subtraction (Vinn)
Focus: HSV thresholding and morphology on `background.avi` and `video.avi`.

In [3]:
def create_background_model(cam_id):
    # TODO: Average frames or use Gaussian mixture
    pass

def get_foreground_mask(frame, bg_model, thresholds):
    # TODO: BGR to HSV -> Diff -> Threshold -> Erode/Dilate
    return mask

# CHOICE 2: Automated thresholding function
def optimize_thresholds(frame, manual_segmentation):
    pass

## Task 3: Voxel Reconstruction (Combined)
Focus: 3D back-projection and lookup tables.

In [4]:
def build_lookup_table(cam_configs, step=10):
    # Programmer A: cv2.projectPoints for voxel grid
    pass

def reconstruct_voxels(masks, lookup_table):
    # Programmer A/B: Logical AND of all camera masks
    pass

# CHOICE 3: Voxel Coloring
# CHOICE 4: Marching Cubes

## Main Pipeline Execution

In [ ]:
if __name__ == "__main__":
    # 1. Run Calibration (Cell 2)
    for cam_id in CAM_IDS:
        mtx, dist = calibrate_intrinsics(cam_id)
        rvec, tvec, handles = calibrate_extrinsics(cam_id, mtx, dist)
        save_all_params(cam_id, mtx, dist, rvec, tvec, handles)
        print(f"Visualizing Origin for Camera {cam_id}...")
        visualize_checkerboard_start(cam_id, mtx, dist, rvec, tvec)
    # 2. Build BG Models (B)
    # 3. Create LUT (A)
    # 4. Process Video frames and render 3D (A&B)
    print("Pipeline Initialized")

--- Extrinsic Calibration: Camera 1 ---
Visualizing Origin for Camera 1...
--- Extrinsic Calibration: Camera 2 ---
Visualizing Origin for Camera 2...
--- Extrinsic Calibration: Camera 3 ---
Visualizing Origin for Camera 3...
--- Extrinsic Calibration: Camera 4 ---
Visualizing Origin for Camera 4...
Pipeline Initialized
